In [1]:
import pandas as pd
import numpy as np

#df = pd.read_csv('~/goinfre/data/train_concat2_90_00001.csv')
#y = pd.read_csv('~/Downloads/y2.csv', sep=';', usecols=[1])
df = pd.read_csv('E:\\Downloads\\train2_irf_00010.csv')
df.apply(pd.to_numeric,downcast='unsigned')
y = pd.read_csv('E:\\Downloads\\y2.csv', sep=';', usecols=[1])
y.apply(pd.to_numeric,downcast='unsigned')

df.shape

(26495, 1204)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp


#param_range = [0.001,0.01,0.1,1,10,100]
#param_range = [0.001]
param_range = [1]

cv = StratifiedKFold(n_splits=3, random_state=42)

for i in param_range:
    aucs = []
    tprs = []
    mean_fpr = np.linspace(0, 1, 100)

    classifier = RandomForestClassifier(n_estimators=200, oob_score=False, n_jobs=-1, min_samples_split=2, min_samples_leaf=1, random_state=42)
    
    for train_index, test_index in cv.split(df, y):
        model = classifier.fit(df.iloc[train_index], y.iloc[train_index].values.ravel())
        if hasattr(model, "predict_proba"):
            probas_ = model.predict_proba(df.iloc[test_index])[:, 1]
        else:  # use decision function
            probas_ = model.decision_function(df.iloc[test_index])
        fpr, tpr, thresholds = roc_curve(y.iloc[test_index], probas_)
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    print('param=%s Mean ROC (AUC = %0.3f)' % (str(i), mean_auc))
    print(aucs)
    

param=1 Mean ROC (AUC = 0.760)
[0.75904965240936617, 0.74573783703823537, 0.77396363177205707]
